In [2]:
import pandas as pd
data = pd.read_csv('/Users/xzhou/github/project_files/amazon/amazon_echo.csv', error_bad_lines=False);
# data_text = data[['headline_text']]
# data_text['index'] = data_text.index
# documents = data_text

Data Pre-processing
We will perform the following steps:
Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
Words that have fewer than 3 characters are removed.
All stopwords are removed.
Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
Words are stemmed — words are reduced to their root form.
Loading gensim and nltk libraries

In [3]:
data.head()

,Pageurl,Title,Review Text,Review Color,User Verified,Review Date,Review Useful Count,Configuration Text,Rating,Declaration Text
0,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Three Stars,Not great speakers,Black,Verified Purchase,10/3/2017,NaN,Echo Dot,3,NaN
1,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Four Stars,Great little gagit,White,Verified Purchase,9/26/2017,NaN,Echo Dot,4,NaN
2,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Awesome!,Awesome 👏🏽,White,Verified Purchase,9/8/2017,NaN,Echo Dot,5,NaN
3,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Five Stars,Love my Echo,Black,Verified Purchase,10/19/2017,NaN,Echo Dot,5,NaN
4,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Five Stars,Great device,Black,Verified Purchase,9/17/2017,NaN,Echo Dot,5,NaN


In [21]:
# Loading gensim and nltk libraries

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/xzhou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# Write a function to perform lemmatize and stem preprocessing steps on the data set.

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [16]:
data.iloc[1, :]

Pageurl                https://www.amazon.com/All-New-Amazon-Echo-Dot...
Title                                                         Four Stars
Review Text                                           Great little gagit
Review Color                                                       White
User Verified                                          Verified Purchase
Review Date                                                    9/26/2017
Review Useful Count                                                  NaN
Configuration Text                                              Echo Dot
Rating                                                                 4
Declaration Text                                                     NaN
Name: 1, dtype: object

In [22]:
# Select a document to preview after preprocessing.

doc_sample = str(data) #documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Pageurl', '', '\\\n0', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n1', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n2', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n3', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n4', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n5', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n6', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n7', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n8', '', '', '', '', 'https://www.amazon.com/All-New-Amazon-Echo-Dot...', '', '', '\n9', '', '',

NameError: name 'stemmer' is not defined

In [23]:
# For each document we create a dictionary reporting how many
# words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

NameError: name 'processed_docs' is not defined

In [ ]:
# For each document we create a dictionary reporting how many
# words and how many times those words appear. Save this to ‘bow_corpus’, 
# then check our selected document earlier.

bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
# Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, 
# then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
# Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
# For each topic, we will explore the words occuring in that topic and its relative weight.

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# Running LDA using TF-IDF

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
# Performance evaluation by classifying sample document using LDA TF-IDF model.

for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
# Testing model on unseen document

unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))